In [1]:
import requests
import cv2 #OpenCV(Open Source Computer Vision Library)의 파이썬 바인딩
import pytesseract
from PIL import Image
import numpy as np
import time
import logging
import json

logging.basicConfig(level=logging.INFO)

def load_config_json(file_path): #Json파일을 로드
    try:
        with open(file_path, 'r') as file:
            config = json.load(file)
        return config
    except FileNotFoundError:
        logging.error(f"파일을 찾을 수 없습니다: {file_path}")
    except json.JSONDecodeError as e:
        logging.error(f"JSON 구문 오류 발생: {e}")
    except Exception as e:
        logging.error(f"오류 발생: {e}")

def fetch_data_with_retries(url, headers, max_retries=5, backoff_factor=1): #API 호출을 재시도하며 데이터를 가져오는 코드
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # HTTP 오류가 발생하면 예외를 던집니다.
            return response.json()  # JSON 응답을 반환합니다.
        except requests.exceptions.HTTPError as err:
            if response.status_code == 429:  # Too Many Requests
                logging.warning(f"요청 제한 초과. {backoff_factor ** retries}초 후 재시도...")
                time.sleep(backoff_factor ** retries)
                retries += 1
            else:
                raise err
    raise Exception("최대 재시도 횟수 초과")

def get_user_num(nick, base_url, headers): #nick으로 userNum추출
    info_url = f"{base_url}user/nickname?query={nick}"
    try:
        info_data = fetch_data_with_retries(info_url, headers) 
        userNum = info_data.get("user", {}).get("userNum")
        return userNum
    except Exception as e:
        logging.error(f"사용자 번호 추출 오류: {e}")

def get_character_info(characterCode, base_url_v2, headers): #캐릭터 정보 추출(DB?)
    try:
        character_url = f"{base_url_v2}data/Character"
        character_data = fetch_data_with_retries(character_url, headers).get("data", [])
        character_info = character_data[characterCode - 1] if characterCode - 1 < len(character_data) else None
        return character_info
    except Exception as e:
        logging.error(f"캐릭터 정보 추출 오류: {e}")   

def get_user_stats(userNum, base_url, headers, seasonId): #userNum으로 seosonID에 해당하는 사용캐릭터 정보추출
    
    stats_url = f"{base_url}user/stats/{userNum}/{seasonId}"
    try:
        stats_data = fetch_data_with_retries(stats_url, headers).get('userStats', [])
        if not stats_data:
            return []
        return [stat for stat in stats_data[0].get('characterStats', []) if 'characterCode' in stat]
        #mostCharacter_list = []
        #character_stats = stats_data[0].get('characterStats', [])
        #for stat in character_stats:
            #if 'characterCode' in stat:
                #mostCharacter_list.append(stat)

        #return(mostCharacter_list)
    except Exception as e:
        logging.error(f"사용자 통계 추출 오류: {e}")

def match_chatacter(nick, base_url, base_url_v2, headers, seasonId): #nick으로 추출한 사용캐릭터 정보와 캐릭터 정보를 매치하여 반환
    userNum = get_user_num(nick, base_url, headers)

    if not userNum: #
        logging.error(f"사용자 {nick}을(를) 찾을 수 없습니다.")
        return None, None #
    
    mostCharacter_list = get_user_stats(userNum, base_url, headers, seasonId)

    if not mostCharacter_list: #
        logging.error(f"사용자 {nick}의 캐릭터 통계를 찾을 수 없습니다.")
        return None, None
    
    character_info_list = [] #
    for mostCharacter in mostCharacter_list:
        characterCode = mostCharacter['characterCode']
        character_info = get_character_info(characterCode, base_url_v2, headers)
        if character_info:
            character_info_list.append(character_info)
        else:
            logging.warning(f"캐릭터 코드 {character_code}에 대한 정보가 없습니다.")
    return character_info_list, mostCharacter_list

def get_user_names(image_path): #Tesseract를 통해 캡쳐된 사진에서 NickList 추출
    nick_list = []
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

    try:
        pil_image = Image.open(image_path)
        image = np.array(pil_image) #PIL 이미지를 NumPy 배열로 변환하면 OpenCV와 같은 이미지 처리 라이브러리와의 호환성, 고속 연산, 배열 조작의 유연성 등의 장점이 있지만, 메모리 사용량 증가와 변환 오버헤드, 형식 차이 등 단점이 있음
        #pil_image = Image.fromarray(image)

        start_len: int = 1080
        start_hgt: int = 930
        nameSpace_interval: int = 284
        nameSpace_len: int = 135
        nameSpace_hgt: int = 20
        #(1080,930,1215,950) 초기값

        for i in range(3):
            crop_area = (start_len+(nameSpace_interval*i),start_hgt,start_len+(nameSpace_interval*i)+nameSpace_len,start_hgt    +nameSpace_hgt) # 특정 영역 자르기 (left, upper, right, lower)
            cropped_image = image[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2]]
            result = pytesseract.image_to_string(cropped_image, lang='kor+eng+jpn')
            if result:
                nick_list.append(result.replace('\n',''))
            else:
                #cv2.imshow(f'Cropped Image {i+1}', cropped_image)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                logging.warning(f"크롭된 이미지 {i+1}에서 텍스트를 추출할 수 없습니다.")
                nick_list.append(None)            
    except Exception as e:
        logging.error(f"이미지 처리 오류: {e}")
    return nick_list

def main():
    config_file = 'C:\\Start\\python_basic\\05.Tesseract\\config.json'
    config = load_config_json(config_file)
    if not config:
        logging.error("설정을 로드할 수 없습니다.")
        return
    
    api_key = config.get("api_key")
    base_url = config.get("base_url")
    base_url_v2 = config.get("base_url_v2")

    if not api_key:
        print("API_KEY가 설정되지 않았습니다.")
        return
    if not base_url:
        print("base_url이 설정되지 않았습니다.")
        return
    if not base_url_v2:
        print("base_url_v2가 설정되지 않았습니다.")
        return
    headers = {"x-api-key": api_key}
    seasonId = 25 #EA9*2 = 18  정규3*2 = 6
    #matchingTeamMode = 3 #스쿼드 3
    image_path = 'C:\\Start\\python_basic\\05.Tesseract\\Image\\test1.jpg'
    nick_list = get_user_names(image_path)
    for nick in nick_list:
        character_info_list, mostCharacter_list = match_chatacter(nick, base_url, base_url_v2, headers, seasonId)
        #print(mostCharacter_list)
        if character_info_list:
            logging.info(f'닉네임: {nick}')
            for idx, character_info in enumerate(character_info_list[:3], 1): #첫 3 개의 요소를 순회하면서, 인덱스를 1부터 시작
                most_character = mostCharacter_list[idx-1] if idx-1 < len(mostCharacter_list) else {}
                logging.info(f'{idx}_캐릭터: {character_info.get("name", "알 수 없음")} '
                    f'({most_character.get("usages", 0)} 게임, {most_character.get("wins", 0)} 승, '
                    f'Top3: {most_character.get("top3", 0)}, 평균 등수: {most_character.get("averageRank", 0)})')

if __name__ == "__main__":
    main()

INFO:root:닉네임: 검성아시나잇신
INFO:root:1_캐릭터: Yuki (82 게임, 18 승, Top3: 40, 평균 등수: 4)
INFO:root:2_캐릭터: Magnus (60 게임, 7 승, Top3: 24, 평균 등수: 4)
INFO:root:3_캐릭터: Charlotte (27 게임, 6 승, Top3: 9, 평균 등수: 4)
INFO:root:닉네임: 23MRFT73
INFO:root:1_캐릭터: Tsubame (296 게임, 42 승, Top3: 102, 평균 등수: 5)
INFO:root:2_캐릭터: Isol (219 게임, 34 승, Top3: 84, 평균 등수: 4)
INFO:root:3_캐릭터: Hyunwoo (187 게임, 29 승, Top3: 82, 평균 등수: 4)
ERROR:root:사용자 None을(를) 찾을 수 없습니다.
